# Projet Final - Classification binaire

- **Description du projet :**

>Ce projet consiste à prédire le parti politique gagnant des élections présidentielles de 2020 aux États-Unis à partir des données socio-démographiques

![US](https://prod-media-eng.dhakatribune.com/uploads/2020/06/bigstock-illustration-of-election-357906857-1592827070539.jpg)

## Table de matières
<ol>
    <li><a href="#prob">Définition du problème</a></li>
    <li><a href="#acqui">Acquisition de données</a></li>
    <li><a href="#cible">Création de la target</a></li>
    <li><a href="#gather">Consititution de données</a></li>
    <li><a href="#save">Sauvegarde de données</a></li>
</ol>

<a id="prob"></a>
# 1. Définition du problème

- L'objectif est de *prédire le parti politique victorieux de chaque état U.S aux élections de 2020 en fonction des d'un ensemble de données socio-démographiques*

- Nous avons à notre disposition plusieurs jeux de données, notamment ceux des résulatats de entre 2008 et 2016 et également ceux de 2022 dont il nous faudrait joindre afin d'obtenir un jeu de données exploitable.

- Nous partirons de rien et essayer de construire progressivement une solution atteignant une très bonne performance

- Il s'agit bien d'un problème de *`Machine Learning Supervisé`* et plus précisement d'une tâche de *`classification binaire`* dont l'objectif est de prédire une variable discrète : *`Parti Républicain`* ou *`Parti Démocrate`*

- La solution sera évaluée en utilisant la métrique de *`F1-Score`* qui est le rapport entre deux métriques *`Precision`* et  *`Recall`* que nous utiliserons également pour évaluer le modèle afin comparer les performances.

<a id="acqui"></a>
## 2. Acquisition de données

- **Importation des packages nécessaires pour le travail et chargment de données**

In [1]:
# Pandas la manipulation de données
import numpy as np
import pandas as pd

In [2]:
# Chargment de données

# Données des élections de 2020
df_20 = pd.read_csv("2020_US_County_Level_Presidential_Results.csv")

# Données des élections de 2008 à 2016
df_08_16 = pd.read_csv("US_County_Level_Presidential_Results_08-16.csv")


# Données sur l'éducation
df_education = pd.read_excel("Education.xls", skiprows=4) # Ne pas afficher les 4 premières lignes

# Données sur l'estimation de la population
df_population = pd.read_excel("PopulationEstimates.xls", skiprows=2) # Ne pas afficher les 2 premières lignes

# Données sur l'estimation de la pauvrété
df_poverty = pd.read_excel("PovertyEstimates.xls", skiprows=4)

# Données sur le chômage
df_chomage = pd.read_excel("Unemployment.xls", skiprows=4)

# L'argument 'skiprows' permet de préciser le nombre de lignes à sauter
# c'est-à-dire à ne pas afficher.

- **Aperçu des données de différents ensemble de données**

In [3]:
# Afficher la taille des données aux élections de 2020
print(f"Nous avons au total {df_20.shape[0]} lignes et {df_20.shape[1]} colonnes\n")

# Afficher les 5 premières lignes des élections de 2020
df_20.head()

Nous avons au total 3152 lignes et 10 colonnes



,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022


<a id="cible"></a>
## 3. Création de la target (`variable cible`)

In [4]:
# Créer la target
df_20["win"] = np.where(df_20["votes_gop"] > df_20["votes_dem"], 
                        1, 0).astype(np.int64)

# Vérification des modifications apportées
df_20.head()

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,win
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184,1
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623,1
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631,1
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280,1
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022,1


In [5]:
# Vérification des modalités de distristribution de la target
print(df_20["win"].value_counts(normalize=True))

1    0.823287
0    0.176713
Name: win, dtype: float64


In [6]:
# Afficher la taille des données élections de 2008 à 2016
print(f"Nous avons au total {df_08_16.shape[0]} lignes et {df_08_16.shape[1]} colonnes\n")

# Afficher les 5 premières lignes des élection de 2008 à 2016
df_08_16.head()

Nous avons au total 3112 lignes et 14 colonnes



,fips_code,county,total_2008,dem_2008,gop_2008,oth_2008,total_2012,dem_2012,gop_2012,oth_2012,total_2016,dem_2016,gop_2016,oth_2016
0,26041,Delta County,19064,9974,8763,327,18043,8330,9533,180,18467,6431,11112,924
1,48295,Lipscomb County,1256,155,1093,8,1168,119,1044,5,1322,135,1159,28
2,1127,Walker County,28652,7420,20722,510,28497,6551,21633,313,29243,4486,24208,549
3,48389,Reeves County,3077,1606,1445,26,2867,1649,1185,33,3184,1659,1417,108
4,56017,Hot Springs County,2546,619,1834,93,2495,523,1894,78,2535,400,1939,196


In [7]:
# Afficher la taille des données d'éducation
print(f"Nous avons au total {df_education.shape[0]} lignes et {df_education.shape[1]} colonnes\n")

# Afficher les 5 premières lignes de données de l'éducation
df_education.head()

Nous avons au total 3283 lignes et 47 colonnes



,FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2015-19","High school diploma only, 2015-19","Some college or associate's degree, 2015-19","Bachelor's degree or higher, 2015-19","Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19"
0,0,US,United States,NaN,NaN,NaN,NaN,52373312.0,34158051.0,11650730.0,...,27.4,24.4,26472261.0,59472748.0,63756905.0,70920162.0,11.998918,26.956844,28.898697,32.145542
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,1062306.0,468269.0,136287.0,...,25.9,19.0,458922.0,1022839.0,993344.0,845772.0,13.819302,30.800268,29.912098,25.468332
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,6611.0,3757.0,933.0,...,26.9,18.0,4291.0,12551.0,10596.0,9929.0,11.483395,33.588459,28.356571,26.571573
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,18726.0,8426.0,2334.0,...,29.3,23.1,13893.0,41797.0,47274.0,48148.0,9.193843,27.659616,31.284081,31.862459
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,8120.0,2242.0,581.0,...,21.3,10.9,4812.0,6396.0,4676.0,2080.0,26.786907,35.604542,26.029837,11.578713


In [8]:
# Afficher la taille des données d'estimation de population
print(f"Nous avons au total {df_population.shape[0]} lignes et {df_population.shape[1]} colonnes\n")

# Afficher les 5 premières lignes de données d'estimation de population
df_population.head()

Nous avons au total 3273 lignes et 165 colonnes



,FIPStxt,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,...,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019
0,0,US,United States,NaN,NaN,NaN,NaN,NaN,308745538,308758105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,NaN,4779736,4780125,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,0.0,54571,54597,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
3,1003,AL,Baldwin County,4.0,3.0,5.0,2.0,5.0,182265,182265,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,3.0,27457,27455,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664


In [9]:
# Afficher la taille des données de pauvrété
print(f"Nous avons au total {df_poverty.shape[0]} lignes et {df_poverty.shape[1]} colonnes\n")

# Afficher les 5 premières lignes de données de pauvrété
df_poverty.head()

Nous avons au total 3193 lignes et 34 colonnes



,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2019,CI90LBALL_2019,CI90UBALL_2019,...,CI90UB517P_2019,MEDHHINC_2019,CI90LBINC_2019,CI90UBINC_2019,POV04_2019,CI90LB04_2019,CI90UB04_2019,PCTPOV04_2019,CI90LB04P_2019,CI90UB04P_2019
0,0,US,United States,NaN,NaN,NaN,NaN,39490096,39248096,39732096,...,16.0,65712,65594,65830,3457689.0,3405854.0,3509524.0,18.2,17.9,18.5
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,747478,730491,764465,...,21.6,51771,51179,52363,69236.0,65296.0,73176.0,24.2,22.8,25.6
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,6723,5517,7929,...,19.4,58233,52517,63949,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,22360,18541,26179,...,17.2,59871,54593,65149,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,5909,4787,7031,...,49.0,35972,31822,40122,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Afficher la taille des données du chômage
print(f"Nous avons au total {df_chomage.shape[0]} lignes et {df_chomage.shape[1]} colonnes\n")

# Afficher les 5 premières lignes de données du chômage 
df_chomage.head()

Nous avons au total 3275 lignes et 88 colonnes



,fips_txt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,0,US,United States,NaN,NaN,NaN,142601667.0,136904680.0,5696987.0,3.995035,...,161389026.0,155102319.0,6286707.0,3.895375,163100055.0,157115247.0,5984808.0,3.669409,65712.0,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,2133223.0,2035594.0,97629.0,4.600000,...,2216627.0,2130845.0,85782.0,3.900000,2241747.0,2174483.0,67264.0,3.000000,51771.0,100.000000
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,21720.0,20846.0,874.0,4.000000,...,26196.0,25261.0,935.0,3.600000,26172.0,25458.0,714.0,2.700000,58233.0,112.481888
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,69533.0,66971.0,2562.0,3.700000,...,95233.0,91809.0,3424.0,3.600000,97328.0,94675.0,2653.0,2.700000,59871.0,115.645828
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,11373.0,10748.0,625.0,5.500000,...,8414.0,7987.0,427.0,5.100000,8537.0,8213.0,324.0,3.800000,35972.0,69.482918


<a id="gather"></a>
## 4. Constitution de l'ensemble de données principal

Maintenant que nos données sont bien chargés dans les différentes tables, il est temps de faire le lien entre ces différentes sources de données afin de bien mener nos analyses.

Pour cela, nous avons choisi la méthode *`merge`* en faisant un *`inner join`* de `DataFrames` en faisant l'intersection des tables sur la clé de jointure considérée, ici c'est la colonne ***`fips_code`*** de chaque table `DataFrame` que nous allons d'abord renomer.

- **Renomer les colonnes permettant de faire la jointure de nos tables dans tous les data sets**

In [11]:
# Renomer la colonne de 'county_fips' en 'fips_code' des élections de 2020
df_20.rename(columns={"county_fips" : "fips_code"}, inplace=True)

# Renomer la colonne 'FIPS Code' en 'fips_code' des données d'éducation
df_education.rename(columns={"FIPS Code" : "fips_code"}, inplace=True)

# Renomer la colonne 'FIPStxt' en 'fips_code' des données d'estimation de la population
df_population.rename(columns={"FIPStxt" : "fips_code"}, inplace=True)

# Renomer la colonne 'FIPStxt' en 'fips_code' des données d'estimation de la pauvrété
df_poverty.rename(columns={"FIPStxt" : "fips_code"}, inplace=True)

# Renomer la colonne 'fips_txt' en 'fips_code' des données de chômage
df_chomage.rename(columns={"fips_txt" : "fips_code"}, inplace=True)

- **Vérification des changements apportés aux colonnes**

In [12]:
# Il suffit de vérifier sur l'un des ensembles de donnéeés
df_20.columns

Index(['state_name', 'fips_code', 'county_name', 'votes_gop', 'votes_dem',
       'total_votes', 'diff', 'per_gop', 'per_dem', 'per_point_diff', 'win'],
      dtype='object')

Nos changements ont bien été apportés, à présent nous pouvons procéder à la consitution de l'ensemble de données unique principal  et exploitable qui servira pour le reste de l'analyse

In [13]:
# Vérifier la taille des données de 2010
df_20.shape

(3152, 11)

In [14]:
# Joindre les deux premières tables des éléctions : 2020 et de 2008 à 2016
election_df = pd.merge(df_20, df_08_16, on="fips_code", how="inner")

# Vérification des changements apportés
print(f"Nous avons au total {election_df.shape[0]} lignes et {election_df.shape[1]} colonnes\n")
election_df.head()

Nous avons au total 3111 lignes et 24 colonnes



,state_name,fips_code,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,...,gop_2008,oth_2008,total_2012,dem_2012,gop_2012,oth_2012,total_2016,dem_2016,gop_2016,oth_2016
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184,...,17403,145,23909,6354,17366,189,24661,5908,18110,643
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623,...,61271,756,84988,18329,65772,887,94090,18409,72780,2901
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631,...,5866,67,11459,5873,5539,47,10390,4848,5431,111
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280,...,6262,83,8391,2200,6131,60,8748,1874,6733,141
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022,...,20389,356,23980,2961,20741,278,25384,2150,22808,426


Avec un nombre de 3111 `lignes` et 354 `variables` retenuesn, c'est un ensemble de données assez grand.

In [15]:
# Joindre avec les données d'estimation de la population
election_df = election_df.merge(df_population, on="fips_code", how="inner")

# Joindre avec les données d'éducation
election_df = election_df.merge(df_education, on="fips_code", how="inner")

# Faire la jointure avec les données d'estimation sur la pauvrété
election_df = election_df.merge(df_poverty, on="fips_code", how="inner")

# Faire la jointure avec les données sur le taux de chômage
election_df = election_df.merge(df_chomage, on="fips_code", how="inner")

# Vérification sur tous les changments apportés
print(f"Nous avons au total {election_df.shape[0]} lignes et {election_df.shape[1]} colonnes\n")
election_df.head()

Nous avons au total 3111 lignes et 354 colonnes



,state_name,fips_code,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184,...,26196.0,25261.0,935.0,3.6,26172.0,25458.0,714.0,2.7,58233.0,112.481888
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623,...,95233.0,91809.0,3424.0,3.6,97328.0,94675.0,2653.0,2.7,59871.0,115.645828
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631,...,8414.0,7987.0,427.0,5.1,8537.0,8213.0,324.0,3.8,35972.0,69.482918
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280,...,8605.0,8268.0,337.0,3.9,8685.0,8419.0,266.0,3.1,47918.0,92.557610
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022,...,25069.0,24201.0,868.0,3.5,25331.0,24655.0,676.0,2.7,52902.0,102.184624


<a id="save"></a>
## 5. Sauvegarde des ensembles des données

Afin de pouvoir réutiliser les mêmes DataFrames, nous allons pouvoir les saugader pour une meilleure utilisation et surtout pour ne plus à les reconstituer dans tous les notebooks.

In [16]:
# Sauvegader le DataFrame des élections de 2020 dont nous avons pu créer la target
df_20.to_csv("elections_2020.csv", index=False) # Ignorer les indexes

# Sauvegarder le DataFrame groupant tous les Datasets
election_df.to_csv("elections.csv", index=False)